In [ ]:
import os
import json

In [ ]:
CURRENT_PATH = os.getcwd()
FILE_DIR = "[Original] DPE Take Home Test"
path = CURRENT_PATH+"/"+FILE_DIR
source_name: str = "source.json"
target_name: str = "target.json"

In [ ]:
def extract_json_data(path: str, file_name: str):
    if path is None or path == "":
        raise ValueError("path variable cannot be null or empty")
    elif file_name is None or file_name == "":
        raise ValueError("file_name variable cannot be null or empty")
    else:
        with open(path+"/"+file_name, mode="r") as file:
            data = json.load(file)
            return data
    

In [ ]:
def log(operation, value, parent_key, after_key):
    return {
        "operation": operation,
        "value": value,
        "parent_key": parent_key,
        "after_key": after_key
    }

In [ ]:
def get_key_list(data, parent=[]):
    key_list = []
    for key, value in data.items():
        key_list.append(".".join(parent)+f".{key}" if len(parent) > 0 else key)
        if type(value) is dict:
            add_parent = parent+[key]
            key_list.extend(get_key_list(value, add_parent))
    return key_list

In [ ]:
def get_values(data, target_key, exclude_dict_value=False):
    separated_scheme = target_key.split(".")
    for key in separated_scheme:
        data = data[key]
        
    if exclude_dict_value:
        if type(data) is dict:
            return None
        return {
        separated_scheme[-1]: data
    }

In [ ]:
def get_after_key(data, target_key):
    parent_key = target_key.rsplit(".", 1)[0]
    relevant_key = [key for key in data if parent_key in key and key.count(".") == target_key.count(".")]
    after_key = relevant_key[relevant_key.index(target_key)-1].split(".")[-1] if relevant_key.index(target_key) > 0 else None
    return after_key


In [ ]:
def get_add_operation(source_data, target_data):
    source_key = get_key_list(source_data)
    target_key = get_key_list(target_data)
    diff_key = sorted(list(set(target_key).difference(set(source_key))))
    index = 0
    output = []

    if len(diff_key) == 0:
        return None

    while index < len(diff_key)-1:
        if diff_key[index] in diff_key[index+1]:
            diff_key.pop(index+1)
            index -= 1
        index += 1
    
    for key in diff_key:
        separated_key = key.split(".")
        operation = "add_new_key"
        parent_key = separated_key[0:-1]
        value = get_values(target_data, key)
        after_key = get_after_key(target_key, key)
        output.append(log(operation, value, parent_key, after_key))
    
    return output

In [ ]:
def get_different_value(source_value, target_value):
    if source_value != target_value:
        return target_value
    else:
        return None


In [ ]:
def get_modified_operation(source_data, target_data):
    source_key = get_key_list(source_data)
    target_key = get_key_list(target_data)
    intersect_key = sorted(list(set(source_key).intersection(set(target_key))))
    output = []

    if len(intersect_key) == 0:
        return None

    for key in intersect_key:
        separated_key = key.split(".")
        operation = "modify_value"
        parent_key = separated_key[0:-1]
        after_key = get_after_key(target_key, key)

        target_value = get_values(target_data, key, exclude_dict_value=True)
        source_value = get_values(source_data, key, exclude_dict_value=True)
        value = get_different_value(source_value, target_value)

        output.append(log(operation, value, parent_key, after_key)) if value is not None else output
    return output

In [ ]:
source_data = extract_json_data(path, source_name)
target_data = extract_json_data(path, target_name)

In [114]:
all_operation = []
all_operation += get_add_operation(source_data, target_data)
all_operation += get_modified_operation(source_data, target_data)

In [116]:
all_operation

[{'operation': 'add_new_key',
  'value': None,
  'parent_key': ['event'],
  'after_key': 'name'},
 {'operation': 'add_new_key',
  'value': None,
  'parent_key': ['event', 'location'],
  'after_key': 'country'},
 {'operation': 'add_new_key',
  'value': None,
  'parent_key': ['event', 'performances', 'supporting', 'second'],
  'after_key': None},
 {'operation': 'modify_value',
  'value': {'country': 'Japan'},
  'parent_key': ['event', 'location'],
  'after_key': 'city'}]